# import

In [ ]:
import numpy as np
import glob
import torch.utils.data
import os
import math
from skimage import io, transform
from PIL import Image
import torch
import torchvision as vision
from torchvision import transforms, datasets
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pickle
!pip install tensorboardX
from tensorboardX import SummaryWriter
import time

     |████████████████████████████████| 124 kB 14.5 MB/s 


# data loader

## single

In [ ]:
class SingleImgDataset(torch.utils.data.Dataset):

    def __init__(self, root_dir, scale_aug=False, rot_aug=False, test_mode=False, \
                 num_models=0, num_views=20):
        self.classnames=['bed','bench','cup','chair','dresser','flower_pot','sofa','stool','table','xbox']
        self.root_dir = root_dir
        self.scale_aug = scale_aug
        self.rot_aug = rot_aug
        self.test_mode = test_mode

        set_ = root_dir.split('/')[-2]
        parent_dir = root_dir.rsplit('/',3)[0]
        self.filepaths = []
        print(parent_dir)
        print(set_)
        for i in range(len(self.classnames)):
            
            all_files = sorted(glob.glob(parent_dir+'/'+self.classnames[i]+'/'+set_+'/*/*.png'))
            if num_models == 0:
                # Use the whole dataset
                self.filepaths.extend(all_files)
            else:
                self.filepaths.extend(all_files[:min(num_models,len(all_files))])

        image_size = (224,224)

        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize(image_size),
            transforms.ToTensor(), 
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])


    def __len__(self):
        return len(self.filepaths)


    def __getitem__(self, idx):
        path = self.filepaths[idx]
        class_name = path.split('/')[-4]
        
        class_id = self.classnames.index(class_name)

        # Use PIL instead
        im = Image.open(self.filepaths[idx]).convert('RGB')
        if self.transform:
            im = self.transform(im)

        return (class_id, im, path)

## multi-views

In [ ]:
class MultiviewImgDataset(torch.utils.data.Dataset):

    def __init__(self, root_dir, scale_aug=False, rot_aug=False, test_mode=False, \
                 num_models=0, num_views=20, shuffle=True):
        self.classnames=['bed','bench','cup','chair','dresser','flower_pot','sofa','stool','table','xbox']
        self.root_dir = root_dir
        self.scale_aug = scale_aug
        self.rot_aug = rot_aug
        self.test_mode = test_mode
        self.num_views = num_views

        set_ = root_dir.split('/')[-2]
        parent_dir = root_dir.rsplit('/',3)[0]
        self.filepaths = []
        for i in range(len(self.classnames)):
            all_files = sorted(glob.glob(parent_dir+'/'+self.classnames[i]+'/'+set_+'/*/*.png'))
            ## Select subset for different number of views
            stride = int(20/self.num_views) # 20 6 4 3 2 1
            all_files = all_files[::stride]

            if num_models == 0:
                # Use the whole dataset
                self.filepaths.extend(all_files)
            else:
                self.filepaths.extend(all_files[:min(num_models,len(all_files))])

        if shuffle==True:
            # permute
            rand_idx = np.random.permutation(int(len(self.filepaths)/num_views))
            filepaths_new = []
            for i in range(len(rand_idx)):
                filepaths_new.extend(self.filepaths[rand_idx[i]*num_views:(rand_idx[i]+1)*num_views])
            self.filepaths = filepaths_new

        image_size = (224,224)
        if self.test_mode:
            self.transform = transforms.Compose([
                transforms.Resize(image_size),                        
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])    
        else:
            self.transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])


    def __len__(self):
        return int(len(self.filepaths)/self.num_views)


    def __getitem__(self, idx):
        path = self.filepaths[idx*self.num_views]
        class_name = path.split('/')[-4]
        class_id = self.classnames.index(class_name)
        # Use PIL instead
        imgs = []
        for i in range(self.num_views):
            im = Image.open(self.filepaths[idx*self.num_views+i]).convert('RGB')
            if self.transform:
                im = self.transform(im)
            imgs.append(im)

        return (class_id, torch.stack(imgs), self.filepaths[idx*self.num_views:(idx+1)*self.num_views])

## multi-views-res

In [ ]:
class MultiviewAndResImgDataset(torch.utils.data.Dataset):
    
    def __init__(self, dataset_norm, dataset_30, dataset_60, shuffle=True):
        self.dataset_norm = dataset_norm
        self.dataset_30 = dataset_30
        self.dataset_60 = dataset_60

    def __getitem__(self, index):
        x_norm = self.dataset_norm[index]
        x_30 = self.dataset_30[index]
        x_60 = self.dataset_60[index]
        return x_norm, x_30, x_60

    def __len__(self):
        return len(self.dataset_norm)


# model

In [ ]:
class Model(nn.Module):

    def __init__(self, name):
        super(Model, self).__init__()
        self.name = name


    def save(self, path, epoch=0):
        complete_path = os.path.join(path, self.name)
        if not os.path.exists(complete_path):
            os.makedirs(complete_path)
        torch.save(self.state_dict(), 
                os.path.join(complete_path, 
                    "model-{}.pth".format(str(epoch).zfill(5))))


    def save_results(self, path, data):
        raise NotImplementedError("Model subclass must implement this method.")
        

    def load(self, path, modelfile=None):
        complete_path = os.path.join(path, self.name)
        if not os.path.exists(complete_path):
            raise IOError("{} directory does not exist in {}".format(self.name, path))

        if modelfile is None:
            model_files = glob.glob(complete_path+"/*")
            mf = max(model_files)
        else:
            mf = os.path.join(complete_path, modelfile)

        self.load_state_dict(torch.load(mf))

import torchvision.models as models
class SVCNN(Model):

    def __init__(self, name, nclasses=10, pretraining=True):
        super(SVCNN, self).__init__(name)

        self.classnames=['bed','bench','cup','chair','dresser','flower_pot','sofa','stool','table','xbox']

        self.nclasses = nclasses
        self.pretraining = pretraining
        
        self.net_1 = models.alexnet(pretrained=self.pretraining).features
        self.net_2 = models.alexnet(pretrained=self.pretraining).classifier   
        self.net_2._modules['6'] = nn.Linear(4096,10)

    def forward(self, x):
        y = self.net_1(x)
        return self.net_2(y.view(y.shape[0],-1))


class MVCNN(Model):

    def __init__(self, name, model, nclasses=10, num_views=20):
        super(MVCNN, self).__init__(name)

        self.classnames=['bed','bench','cup','chair','dresser','flower_pot','sofa','stool','table','xbox']

        self.nclasses = nclasses
        self.num_views = num_views

        self.net_1 = model.net_1
        self.net_2 = model.net_2

    def forward(self, x):
        y = self.net_1(x)
        y = y.view((int(x.shape[0]/self.num_views),self.num_views,y.shape[-3],y.shape[-2],y.shape[-1]))#(8,20,512,7,7)
        return self.net_2(torch.max(y,1)[0].view(y.shape[0],-1))

    def extract_features(self, x):
        """ Returns output of the FC7 """
        y = self.net_1(x)
        y = y.view((int(x.shape[0]/self.num_views),self.num_views,y.shape[-3],y.shape[-2],y.shape[-1]))#(8,20,512,7,7)
        y = self.net_2._modules['0'](torch.max(y,1)[0].view(y.shape[0],-1))
        y = self.net_2._modules['1'](y)
        y = self.net_2._modules['2'](y)
        y = self.net_2._modules['3'](y)
        y = self.net_2._modules['4'](y)
        # y = self.net_2._modules['5'](y)
        return self.net_2._modules['5'](y)

class MVCNN_RES(Model):
    def __init__(self, name, model_norm, model_30, model_60, nclasses=10, num_views=20):
        super(MVCNN_RES, self).__init__(name)

        self.classnames=['bed','bench','cup','chair','dresser','flower_pot','sofa','stool','table','xbox']

        self.nclasses = nclasses
        self.num_views = num_views

        self.net_1_norm = model_norm.net_1
        self.net_2 = model_norm.net_2

        self.net_1_30 = model_30.net_1
       
        self.net_1_60 = model_60.net_1
        

    def forward(self, x_norm, x_30, x_60):
        y1 = self.net_1_norm(x_norm)
        y2 = self.net_1_30(x_30)
        y3 = self.net_1_60(x_60)
        
        y1 = y1.view((int(x_norm.shape[0]/self.num_views),self.num_views,y1.shape[-3],y1.shape[-2],y1.shape[-1]))#(8,20,512,7,7)
        print(y1.shape)
        y2 = y2.view((int(x_norm.shape[0]/self.num_views),self.num_views,y2.shape[-3],y2.shape[-2],y2.shape[-1]))#(8,20,512,7,7)
        y3 = y3.view((int(x_norm.shape[0]/self.num_views),self.num_views,y3.shape[-3],y3.shape[-2],y3.shape[-1]))#(8,20,512,7,7)
        y = torch.cat([y1, y2, y3],dim=1)

        return self.net_2(torch.max(y,1)[0].view(y.shape[0],-1))

# train

## train for MVCNN/SVCNN

In [ ]:
class ModelNetTrainer(object):

    def __init__(self, model, train_loader, val_loader, optimizer, loss_fn, \
                 model_name, log_dir, num_views=20):

        self.optimizer = optimizer
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.loss_fn = loss_fn
        self.model_name = model_name
        self.log_dir = log_dir
        self.num_views = num_views

        self.model.cuda()
        if self.log_dir is not None:
            self.writer = SummaryWriter(log_dir)


    def train(self, n_epochs):

        best_acc = 0
        i_acc = 0
        self.model.train()
        for epoch in range(n_epochs):
            # permute data for mvcnn
            rand_idx = np.random.permutation(int(len(self.train_loader.dataset.filepaths)/self.num_views))
            filepaths_new = []
            for i in range(len(rand_idx)):
                filepaths_new.extend(self.train_loader.dataset.filepaths[rand_idx[i]*self.num_views:(rand_idx[i]+1)*self.num_views])
            self.train_loader.dataset.filepaths = filepaths_new

            # plot learning rate
            lr = self.optimizer.state_dict()['param_groups'][0]['lr']
            self.writer.add_scalar('params/lr', lr, epoch)

            # train one epoch
            out_data = None
            in_data = None
            for i, data in enumerate(self.train_loader):

                if self.model_name == 'mvcnn':
                    N,V,C,H,W = data[1].size()
                    in_data = Variable(data[1]).view(-1,C,H,W).cuda()
                else:
                    in_data = Variable(data[1].cuda())
                target = Variable(data[0]).cuda().long()

                self.optimizer.zero_grad()

                out_data = self.model(in_data)

                loss = self.loss_fn(out_data, target)
                
                self.writer.add_scalar('train/train_loss', loss, i_acc+i+1)

                pred = torch.max(out_data, 1)[1]
                results = pred == target
                correct_points = torch.sum(results.long())

                acc = correct_points.float()/results.size()[0]
                self.writer.add_scalar('train/train_overall_acc', acc, i_acc+i+1)

                loss.backward()
                self.optimizer.step()
                
                log_str = 'epoch %d, step %d: train_loss %.3f; train_acc %.3f' % (epoch+1, i+1, loss, acc)
                if (i+1)%1==0:
                    print(log_str)
            i_acc += i

            # evaluation
            if (epoch+1)%1==0:
                with torch.no_grad():
                    loss, val_overall_acc, val_mean_class_acc = self.update_validation_accuracy(epoch)
                self.writer.add_scalar('val/val_mean_class_acc', val_mean_class_acc, epoch+1)
                self.writer.add_scalar('val/val_overall_acc', val_overall_acc, epoch+1)
                self.writer.add_scalar('val/val_loss', loss, epoch+1)

            # save best model
            if val_overall_acc > best_acc:
                best_acc = val_overall_acc
                self.model.save(self.log_dir, epoch)
 
            # adjust learning rate manually
            if epoch > 0 and (epoch+1) % 10 == 0:
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = param_group['lr']*0.5

        # export scalar data to JSON for external processing
        self.writer.export_scalars_to_json(self.log_dir+"/all_scalars.json")
        self.writer.close()

    def update_validation_accuracy(self, epoch):
        all_correct_points = 0
        all_points = 0

        # in_data = None
        # out_data = None
        # target = None

        wrong_class = np.zeros(10)
        samples_class = np.zeros(10)
        all_loss = 0

        self.model.eval()

        avgpool = nn.AvgPool1d(1, 1)

        total_time = 0.0
        total_print_time = 0.0
        all_target = []
        all_pred = []

        for _, data in enumerate(self.val_loader, 0):

            if self.model_name == 'mvcnn':
                N,V,C,H,W = data[1].size()
                in_data = Variable(data[1]).view(-1,C,H,W).cuda()
            else:#'svcnn'
                in_data = Variable(data[1]).cuda()
            target = Variable(data[0]).cuda()

            out_data = self.model(in_data)
            pred = torch.max(out_data, 1)[1]
            all_loss += self.loss_fn(out_data, target).cpu().data.numpy()
            results = pred == target

            for i in range(results.size()[0]):
                if not bool(results[i].cpu().data.numpy()):
                    wrong_class[target.cpu().data.numpy().astype('int')[i]] += 1
                samples_class[target.cpu().data.numpy().astype('int')[i]] += 1
            correct_points = torch.sum(results.long())

            all_correct_points += correct_points
            all_points += results.size()[0]

        print ('Total # of test models: ', all_points)
        val_mean_class_acc = np.mean((samples_class-wrong_class)/samples_class)
        acc = all_correct_points.float() / all_points
        val_overall_acc = acc.cpu().data.numpy()
        loss = all_loss / len(self.val_loader)

        print ('val mean class acc. : ', val_mean_class_acc)
        print ('val overall acc. : ', val_overall_acc)
        print ('val loss : ', loss)

        self.model.train()

        return loss, val_overall_acc, val_mean_class_acc

## train for multi-input MVCNN-RES

In [ ]:
class ModelMutiNetTrainer(object):

    def __init__(self, model, train_loader, val_loader, optimizer, loss_fn, \
                 model_name, log_dir, num_views=20, num_res=3):
        # input trained mvcnn for 3 res-data separately  
        self.model = model

        # load data
        self.train_loader = train_loader
        self.val_loader = val_loader

        # setting for trainer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

        # logging
        self.model_name = model_name
        self.log_dir = log_dir

        # detail for dataset
        self.num_views = num_views
        self.num_res = num_res

        # using GPU 
        self.device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        if self.log_dir is not None:
            self.writer = SummaryWriter(log_dir)


    def train(self, n_epochs):

        best_acc = 0
        i_acc = 0
        for epoch in range(n_epochs):

            # permute data for mvcnn
            rand_idx = np.random.permutation(int(len(self.train_loader.dataset.dataset_norm.filepaths)/self.num_views))
            filepaths_new_norm = []
            filepaths_new_30 = []
            filepaths_new_60 = []
            for i in range(len(rand_idx)):
                filepaths_new_norm.extend(self.train_loader.dataset.dataset_norm.filepaths[rand_idx[i]*self.num_views:(rand_idx[i]+1)*self.num_views])
                filepaths_new_30.extend(self.train_loader.dataset.dataset_30.filepaths[rand_idx[i]*self.num_views:(rand_idx[i]+1)*self.num_views])
                filepaths_new_60.extend(self.train_loader.dataset.dataset_60.filepaths[rand_idx[i]*self.num_views:(rand_idx[i]+1)*self.num_views])
            self.train_loader.dataset.dataset_norm.filepaths = filepaths_new_norm
            self.train_loader.dataset.dataset_30.filepaths = filepaths_new_30
            self.train_loader.dataset.dataset_60.filepaths = filepaths_new_60

            # plot learning rate
            lr = self.optimizer.state_dict()['param_groups'][0]['lr']
            self.writer.add_scalar('params/lr', lr, epoch)

            # train one epoch
            out_data = None
            in_data = None
            for i, data in enumerate(self.train_loader):

                N,V,C,H,W = data[0][1].size()
                in_data_norm = Variable(data[0][1]).view(-1,C,H,W).to(device)
                in_data_30 = Variable(data[1][1]).view(-1,C,H,W).to(device)
                in_data_60 = Variable(data[2][1]).view(-1,C,H,W).to(device)

                target = Variable(data[0][0]).to(device).long()

                self.optimizer.zero_grad()

                out_data = self.model(in_data_norm, in_data_30, in_data_60)

                loss = self.loss_fn(out_data, target)
                
                self.writer.add_scalar('train/train_loss', loss, i_acc+i+1)

                pred = torch.max(out_data, 1)[1]
                results = pred == target
                correct_points = torch.sum(results.long())

                acc = correct_points.float()/results.size()[0]
                self.writer.add_scalar('train/train_overall_acc', acc, i_acc+i+1)

                loss.backward()
                self.optimizer.step()
                
                log_str = 'epoch %d, step %d: train_loss %.3f; train_acc %.3f' % (epoch+1, i+1, loss, acc)
                if (i+1)%1==0:
                    print(log_str)
            i_acc += i

            # evaluation
            if (epoch+1)%1==0:
                with torch.no_grad():
                    loss, val_overall_acc, val_mean_class_acc = self.update_validation_accuracy(epoch)
                self.writer.add_scalar('val/val_mean_class_acc', val_mean_class_acc, epoch+1)
                self.writer.add_scalar('val/val_overall_acc', val_overall_acc, epoch+1)
                self.writer.add_scalar('val/val_loss', loss, epoch+1)

            # save best model
            if val_overall_acc > best_acc:
                best_acc = val_overall_acc
                self.model.save(self.log_dir, epoch)
 
            # adjust learning rate manually
            if epoch > 0 and (epoch+1) % 10 == 0:
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = param_group['lr']*0.5

        # export scalar data to JSON for external processing
        self.writer.export_scalars_to_json(self.log_dir+"/all_scalars.json")
        self.writer.close()

    def update_validation_accuracy(self, epoch):
        all_correct_points = 0
        all_points = 0

        # in_data = None
        # out_data = None
        # target = None

        wrong_class = np.zeros(10)
        samples_class = np.zeros(10)
        all_loss = 0

        self.model.eval()

        avgpool = nn.AvgPool1d(1, 1)

        total_time = 0.0
        total_print_time = 0.0
        all_target = []
        all_pred = []

        for _, data in enumerate(self.val_loader, 0):
            N,V,C,H,W = data[0][1].size()
            in_data_norm = Variable(data[0][1]).view(-1,C,H,W).to(device)
            in_data_30 = Variable(data[1][1]).view(-1,C,H,W).to(device)
            in_data_60 = Variable(data[2][1]).view(-1,C,H,W).to(device)

            target = Variable(data[0][0]).to(device)


            out_data = self.model(in_data_norm, in_data_30, in_data_60)

            pred = torch.max(out_data, 1)[1]
            all_loss += self.loss_fn(out_data, target).cpu().data.numpy()
            results = pred == target

            for i in range(results.size()[0]):
                if not bool(results[i].cpu().data.numpy()):
                    wrong_class[target.cpu().data.numpy().astype('int')[i]] += 1
                samples_class[target.cpu().data.numpy().astype('int')[i]] += 1
            correct_points = torch.sum(results.long())

            all_correct_points += correct_points
            all_points += results.size()[0]

        print ('Total # of test models: ', all_points)
        val_mean_class_acc = np.mean((samples_class-wrong_class)/samples_class)
        acc = all_correct_points.float() / all_points
        val_overall_acc = acc.cpu().data.numpy()
        loss = all_loss / len(self.val_loader)

        print ('val mean class acc. : ', val_mean_class_acc)
        print ('val overall acc. : ', val_overall_acc)
        print ('val loss : ', loss)

        self.model.train()

        return loss, val_overall_acc, val_mean_class_acc

## training for MVCNN-RES

In [ ]:
import torch.optim as optim
model = MVCNN_RES('MVCNN-RES', model_2_norm, model_2_30, model_2_60)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0, betas=(0.9, 0.999))
trainer = ModelMutiNetTrainer(model, train_loader_multi, test_loader_multi, optimizer, nn.CrossEntropyLoss(), 'MVCNN-RES', 'MVCNN-RES', num_views=20)
trainer.train(3)

# CNN-feather

## test for tensor to pandas

In [ ]:
import torch
import pandas as  pd

x1 = torch.rand(4,4)
x2 = torch.rand(4,4)
y = torch.rand(8,1)
px = pd.DataFrame(x1.numpy())
px = px.append(pd.DataFrame(x2.numpy()), ignore_index=True)
px['label'] = y.numpy()
px

,0,1,2,3,label
0,0.261833,0.111066,0.692498,0.743136,0.375229
1,0.100112,0.144430,0.947567,0.316236,0.376510
2,0.579914,0.612485,0.760884,0.346794,0.814628
3,0.344728,0.113983,0.637992,0.417706,0.293498
4,0.512333,0.367342,0.956675,0.285753,0.314751
5,0.522563,0.021251,0.202982,0.395416,0.017155
6,0.800201,0.134736,0.866983,0.012809,0.190610
7,0.331038,0.346326,0.228763,0.319438,0.585658


## get_feather(name, train_dataset, train_loader, model)

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import pandas as  pd
def get_feather(name, train_dataset, train_loader, model):

  model=model.to(device)
  nb_features = 4096
  lens = int(len(train_dataset.filepaths)/8/20)
  for i, data in enumerate(train_loader):
      N,V,C,H,W = data[1].size()
      in_data = Variable(data[1]).view(-1,C,H,W).to(device)
      out_data = model.extract_features(in_data).data
      feather = out_data.cpu().detach().numpy()
      target = data[0].numpy()
      if i == 0:
        p_feather = pd.DataFrame(feather)
        labels = pd.DataFrame(target)
      else:
        p_feather = p_feather.append(pd.DataFrame(feather), ignore_index=True)
        labels = labels.append(pd.DataFrame(target), ignore_index=True)
      if i % 10 == 0 or i == lens:
        print(i)
        p_feather.to_csv(name + 'CNN-feather.csv')
        labels.to_csv(name + 'CNN-feather-labels.csv')

  print('CNN features obtained and saved.')

# connect google 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# load trained mvcnn model

In [26]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model_1 = SVCNN("svcnn")
model_2_norm = MVCNN('MVCNN',model=model_1).to(device)
model_2_30 = MVCNN('MVCNN',model=model_1)
model_2_60 = MVCNN('MVCNN',model=model_1)

model_weight_path_norm = torch.load('/content/drive/MyDrive/mvcnn/running_cache5_stage_2/running_cache5/model-00002.pth', map_location=device)
model_2_norm.load_state_dict(model_weight_path_norm)

model_weight_path_30 = torch.load('/content/drive/MyDrive/mvcnn/running_cache30_stage_2/running_cache30/model-00001.pth')
model_2_30.load_state_dict(model_weight_path_30)

model_weight_path_60 = torch.load('/content/drive/MyDrive/mvcnn/running_cache60_stage_2/running_cache60/model-00000.pth', map_location=device)
model_2_60.load_state_dict(model_weight_path_60)


cpu


<All keys matched successfully>

# load data

## train data

In [24]:
train_path_norm = '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/*/train/*'
train_dataset_norm = MultiviewImgDataset(train_path_norm, scale_aug=False, rot_aug=False, shuffle=False, num_models=0, num_views=20)
train_loader_norm = torch.utils.data.DataLoader(train_dataset_norm, batch_size=8, shuffle=False, num_workers=0)
print('num_train_files(norm): '+str(len(train_dataset_norm.filepaths)))

train_path_30 = '/content/drive/MyDrive/data/ModelNet4_voxel_20views_30/*/train/*'
train_dataset_30 = MultiviewImgDataset(train_path_30, scale_aug=False, rot_aug=False, shuffle=False, num_models=0, num_views=20)
train_loader_30 = torch.utils.data.DataLoader(train_dataset_30, batch_size=8, shuffle=False, num_workers=0)
print('num_train_files(30 res): '+str(len(train_dataset_30.filepaths)))

train_path_60 = '/content/drive/MyDrive/data/ModelNet4_voxel_20views_60/*/train/*'
train_dataset_60 = MultiviewImgDataset(train_path_60, scale_aug=False, rot_aug=False, shuffle=False, num_models=0, num_views=20)
train_loader_60 = torch.utils.data.DataLoader(train_dataset_60, batch_size=8, shuffle=False, num_workers=0)
print('num_train_files(60 res): '+str(len(train_dataset_60.filepaths)))

num_train_files(30 res): 31840
num_train_files(60 res): 31840


## test data

In [23]:
test_path_norm = '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/*/test/*'
test_dataset_norm = MultiviewImgDataset(test_path_norm, scale_aug=False, rot_aug=False, shuffle=False, num_models=0, num_views=20)
test_loader_norm = torch.utils.data.DataLoader(test_dataset_norm, batch_size=8, shuffle=False, num_workers=0)
print('num_test_files(norm): '+str(len(test_dataset_norm.filepaths)))

test_path_30 = '/content/drive/MyDrive/data/ModelNet4_voxel_20views_30/*/test/*'
test_dataset_30 = MultiviewImgDataset(test_path_30, scale_aug=False, rot_aug=False, shuffle=False, num_models=0, num_views=20)
test_loader_30 = torch.utils.data.DataLoader(test_dataset_30, batch_size=8, shuffle=False, num_workers=0)
print('num_test_files(30 res): '+str(len(test_dataset_30.filepaths)))

test_path_60 = '/content/drive/MyDrive/data/ModelNet4_voxel_20views_60/*/test/*'
test_dataset_60 = MultiviewImgDataset(test_path_60, scale_aug=False, rot_aug=False, shuffle=False, num_models=0, num_views=20)
test_loader_60 = torch.utils.data.DataLoader(test_dataset_60, batch_size=8, shuffle=False, num_workers=0)
print('num_test_files(60 res): '+str(len(test_dataset_60.filepaths)))

num_test_files(30 res): 4460
num_test_files(60 res): 4460


# get CNN feather

In [ ]:
# get cnn feather for norm
get_feather('norm', train_dataset_norm, train_loader_norm, model_2_norm)

# get cnn feather for 30
get_feather('30', train_dataset_30, train_loader_30, model_2_30)

# get cnn feather for 60
get_feather('60', train_dataset_60, train_loader_60, model_2_60)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
CNN features obtained and saved.


In [25]:
# get cnn feather for norm
get_feather('norm-test', test_dataset_norm, test_loader_norm, model_2_norm)

# get cnn feather for 60
get_feather('60-test', test_dataset_60, test_loader_60, model_2_60)

# get cnn feather for 30
get_feather('30-test', test_dataset_30, test_loader_30, model_2_30)

0
10
20
27
CNN features obtained and saved.


#Test

## test for dataloader（loading order）

In [ ]:
print(train_dataset_norm.filepaths[0:20])

['/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_1.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_10.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_11.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_12.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_13.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_14.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_15.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_16.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_17.png', '/content/drive/MyDrive/data/ModelNet4_voxel_20views_norm/bed/train/bed_0001.off/bed_0001_18.png', '/content/

In [ ]:
train_dataset_multi = MultiviewAndResImgDataset(train_dataset_norm, train_dataset_30, train_dataset_60)
train_loader_multi = torch.utils.data.DataLoader(train_dataset_multi, batch_size=8, shuffle=False, num_workers=0)

## test for data[0] , data.shape

In [ ]:
import pandas as pd

for i, data in enumerate(train_loader_norm):
  if i == 0:
    print(data[0])
    print(data[0].shape)
    p_feather = pd.DataFrame(data[0].numpy())
    print(p_feather)
    break


tensor([0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([8])
   0
0  0
1  0
2  0
3  0
4  0
5  0
6  0
7  0


In [ ]:
get_feather('norm', train_dataset_norm, train_loader_norm, model_2_norm)

In [ ]:
len(labels)

110